### Imports

In [ ]:
from agents.fact_checker_agent import FactCheckerAgent
from network.socialnetwork import SocialNetworkEnv
from agents.news_agent import NewsAgent
import numpy as np

### Setup

In [ ]:
network = SocialNetworkEnv(numConsumer=5)

# created a fake and real info agent and add a few consumers to the network
fAgent = NewsAgent("fake-information", state_space_size=2, env=network, trustLevel=0)
rAgent = NewsAgent("real-information", state_space_size=2, env=network, trustLevel=0)
fcAgent = FactCheckerAgent("fact-checker", trustLevel=0.0, env=network)
network.add_news_agents_to_network(agentType=fAgent)
network.add_news_agents_to_network(agentType=rAgent)
network.add_fact_checker_to_network(agentType=fcAgent)

action = network.draw_sample_from_action_space()
# network.render()

# print("---------------- Initial Graph ---------------")
network.print_graph()

### Training

In [ ]:
for i in range(network.network_size):
    act2 = network.draw_sample_from_action_space()
    print("ASD", act2)
    obs2, reward2, info2 = network.step(act2, rAgent)
    act1 = network.draw_sample_from_action_space()
    obs, reward, info = network.step(act1, fAgent)
    network.render()
    # network.print_graph()

    print(f"--------- epoch {i} ---------------------")
    print("\n\n ------------fake info -------------")
    print("Observation:", obs)
    print("Reward:", reward)
    print("Info:", info)

    print("\n\n ------------real info -------------")
    print("Observation:", obs2)
    print("Reward:", reward2)
    print("Info:", info2)
